# LendingClub - Logistic Regression 

In [ ]:
loans=read.csv("../Csv_Files/loans.csv")

In [ ]:
str(loans)

In [ ]:
summary(loans)

In [ ]:
library(mice)

#### Fill the missing values for all the attributes except class

In [ ]:
fill_feature = setdiff(names(loans),"not.fully.paid")

#### Fill the missed values based on the available non missing variables. Ex: If it is raining, missing temperature value will be less. (Multivariate Imputation by Chained Equations in R)

In [ ]:
complete_feature = complete(mice(loans[fill_feature]))

In [ ]:
loans[fill_feature]= complete_feature

#### Feature with different levels are reduced to same value range. Features with higher range values should not dominate the ones with lower value

In [ ]:
scale_features = setdiff(names(loans),c("not.fully.paid","purpose"))

In [ ]:
scaled_features=scale(loans[scale_features])

In [ ]:
set.seed(144)

In [ ]:
library(caTools)

#### Split the data into training and test data

In [ ]:
split = sample.split(loans$not.fully.paid, SplitRatio = 0.75)

In [ ]:
Train = subset(loans, split == TRUE)

In [ ]:
Test = subset(loans, split == FALSE)

In [ ]:
loanTrain = read.csv("../Csv_Files/loanTrain.csv")

In [ ]:
loanTest = read.csv("../Csv_Files/loanTest.csv")

#### Create Linear regression model

In [ ]:
Model = glm( not.fully.paid~., data=loanTrain, family=binomial)

In [ ]:
summary(Model)

In [ ]:
loanTest$predict.test = predict(Model, type="response", newdata=loanTest)

#### Predict the output of loanTest

In [ ]:
table(loanTest$not.fully.paid,as.numeric(loanTest$predict.test>0.5))

###                        Feature understanding

#### In the summary function, int.rate is not influencial. So, features may be correlated.

In [ ]:
correlationMatrix <- cor(loans[scale_features])

#### Find the highly correlated feature

In [ ]:
library(caret)

In [ ]:
highlyCorrelated <- findCorrelation(correlationMatrix, cutoff=0.5,verbose = TRUE)

In [ ]:
scale_features[6]

In [ ]:
 Model = glm( not.fully.paid~credit.policy+purpose+int.rate+installment+log.annual.inc+dti+days.with.cr.line+revol.bal+revol.util+inq.last.6mths+delinq.2yrs +pub.rec, data=loanTrain, family=binomial)

In [ ]:
loanTest$predict.test = predict(Model, type="response", newdata=loanTest)

In [ ]:
table(loanTest$not.fully.paid,as.numeric(loanTest$predict.test>0.5))

#### Slight improvement in the model

#### Selecting feature smartly will improve your prediction

### Understand response function

 ####    Change the G function manually

#### Find the coefficient of the model

In [ ]:
betas <- coef(Model)

In [ ]:
betas

#### Convert dataframe into matrix (for multiplication)

In [ ]:
para=c()
for(i in seq(1,length(betas)))
{
    para[i]= betas[i]
}

In [ ]:
X <- model.matrix(not.fully.paid~credit.policy+purpose+int.rate+installment+log.annual.inc+dti+days.with.cr.line+revol.bal+revol.util+inq.last.6mths+delinq.2yrs +pub.rec, data=loanTest)

#### Extract the probability of the class

In [ ]:
prob_class <- 1 / (1 +  exp(-(X %*% para)))

In [ ]:
table(loanTest$not.fully.paid,as.numeric(prob_class>0.5))

In [ ]:
prob_class <- 1 / (1 +  exp(-(X %*% para))/2)

In [ ]:
table(loanTest$not.fully.paid,as.numeric(prob_class>0.5))

#### Understanding g function may improve the prediction

In [ ]:
library(gplots)

In [ ]:
library(ROCR)

#### Plotting the precision and specifivity will give better prediction

In [ ]:
pred = prediction(prob_class,loanTest$not.fully.paid)

In [ ]:
ROCRperf = performance(pred, "ppv", "tpr")

In [ ]:
plot(ROCRperf)

In [ ]:
plot(ROCRperf, colorize=TRUE, print.cutoffs.at=seq(0,1,by=0.1), text.adj=c(-0.2,1.7))